# Exercise 6

## SVM & Regularization


For this homework we consider a set of observations on a number of red and white wine varieties involving their chemical properties and ranking by tasters. Wine industry shows a recent growth spurt as social drinking is on the rise. The price of wine depends on a rather abstract concept of wine appreciation by wine tasters, opinion among whom may have a high degree of variability. Pricing of wine depends on such a volatile factor to some extent. Another key factor in wine certification and quality assessment is physicochemical tests which are laboratory-based and takes into account factors like acidity, pH level, presence of sugar and other chemical properties. For the wine market, it would be of interest if human quality of tasting can be related to the chemical properties of wine so that certification and quality assessment and assurance process is more controlled.

Two datasets are available of which one dataset is on red wine and have 1599 different varieties and the other is on white wine and have 4898 varieties. All wines are produced in a particular area of Portugal. Data are collected on 12 different properties of the wines one of which is Quality, based on sensory data, and the rest are on chemical properties of the wines including density, acidity, alcohol content etc. All chemical properties of wines are continuous variables. Quality is an ordinal variable with possible ranking from 1 (worst) to 10 (best). Each variety of wine is tasted by three independent tasters and the final rank assigned is the median rank given by the tasters.

A predictive model developed on this data is expected to provide guidance to vineyards regarding quality and price expected on their produce without heavy reliance on volatility of wine tasters.

In [2]:
import pandas as pd
import numpy as np

In [3]:
data_r = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_red.csv')
data_w = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_white.csv')

In [4]:
data = data_w.assign(type = 'white')

data = data.append(data_r.assign(type = 'red'), ignore_index=True)
data.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
4060,6.4,0.41,0.01,6.1,0.048,20.0,70.0,0.99362,3.19,0.42,10.00,5,white
3528,6.9,0.75,0.13,6.3,0.036,19.0,50.0,0.99312,3.09,0.25,11.10,4,white
5397,8.7,0.69,0.31,3.0,0.086,23.0,81.0,1.00020,3.48,0.74,11.60,6,red
3736,6.5,0.32,0.48,8.0,0.026,18.0,88.0,0.99144,3.22,0.79,12.70,4,white
4684,6.0,0.24,0.34,1.0,0.036,52.0,184.0,0.99097,3.44,0.44,11.45,6,white


# Exercise 6.1

Show the frecuency table of the quality by type of wine

In [5]:
tabla_de_frecuencia = pd.DataFrame(data['quality'].groupby(data['type']).value_counts())
tabla_de_frecuencia

quality
type  quality         
red   5            681
      6            638
      7            199
      4             53
      8             18
      3             10
white 6           2198
      5           1457
      7            880
      8            175
      4            163
      3             20
      9              5

In [23]:
#tab= pivot_table(data, values='quality', index='quality', columns=['type'],  aggfunc='count')

# SVM

# Exercise 6.2

* Standarized the features (not the quality)
* Create a binary target for each type of wine
* Create two Linear SVM's for the white and red wines, repectively.


In [18]:
data['quality_good'] = data["quality"] >= 6 

In [35]:
data_r = data[data.type == 'red' ]
data_r.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type,quality_good
4898,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red,False
4899,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,red,False
4900,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,red,False
4901,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,red,True
4902,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,red,False


In [36]:
data_w = data[data.type == 'white']
data_w.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type,quality_good
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,white,True
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,white,True
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,white,True
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white,True
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white,True


In [37]:
tabla_de_frecuencia2 = pd.DataFrame(data['quality_good'].groupby(data['type']).value_counts())
data['quality_good'].value_counts()

True     4113
False    2384
Name: quality_good, dtype: int64

In [38]:
variables = data.columns.values.tolist()
variables.remove('quality')
variables.remove('quality_good')
variables.remove('type')

variables_y = ['quality','quality_good']

# Tener claro cuáles serán las X's
variables

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [39]:
# Dividir X e Y
X_w = data_w[variables]
y_w = data_w[variables_y]

X_r = data_r[variables]
y_r = data_r[variables_y]

# Aquí para dividir en train y test
from sklearn.model_selection import train_test_split
X_r_train, X_r_test, y_r_train, y_r_test = train_test_split(X_r, y_r)
X_w_train, X_w_test, y_w_train, y_w_test = train_test_split(X_w, y_w)

# Normalizar datos
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(X_r_train)
scaler.fit(X_w_train)
X_w_train_scaled = scaler.transform(X_w_train)
X_w_test_scaled = scaler.transform(X_w_test)

X_r_train_scaled = scaler.transform(X_r_train)
X_r_test_scaled = scaler.transform(X_r_test)

In [40]:
from sklearn.svm import SVC # "Support Vector Classifier"
clf_lin = SVC(kernel='linear', gamma='auto')

In [41]:
y_r_train['quality_good'].head()

6278     True
5690     True
5738     True
5522    False
5662     True
Name: quality_good, dtype: bool

In [42]:
clf_lin.fit(X_r_train_scaled,y_r_train['quality_good'])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [44]:
y_r_pred_lin = clf_lin.predict(X_r_test_scaled)

In [47]:
from sklearn.metrics import accuracy_score
accuracy_score(y_r_test['quality_good'], y_r_pred_lin)

0.74

In [48]:
clf_lin.fit(X_w_train_scaled,y_w_train['quality_good'])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [49]:
y_w_pred_lin = clf_lin.predict(X_w_test_scaled)

In [50]:
accuracy_score(y_w_test['quality_good'], y_w_pred_lin)

0.7404081632653061

# Exercise 6.3

Test the two SVM's using the different kernels (‘poly’, ‘rbf’, ‘sigmoid’)


In [55]:
clf_poly = SVC(kernel='poly', gamma='auto')
clf_rbf = SVC(kernel='rbf', gamma='auto')
clf_sig = SVC(kernel='sigmoid', gamma='auto')

In [56]:
clf_poly.fit(X_r_train_scaled, y_r_train['quality_good'])
clf_rbf.fit(X_r_train_scaled, y_r_train['quality_good'])
clf_sig.fit(X_r_train_scaled, y_r_train['quality_good'])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='sigmoid',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [58]:
y_r_pred_poly = clf_poly.predict(X_r_test_scaled)
y_r_pred_rbf = clf_rbf.predict(X_r_test_scaled)
y_r_pred_sig = clf_sig.predict(X_r_test_scaled)
print(f"Accuracy Score para kernel 'poly' es: {accuracy_score(y_r_test['quality_good'], y_r_pred_poly)}")
print(f"Accuracy Score para kernel 'rbf' es: {accuracy_score(y_r_test['quality_good'], y_r_pred_rbf)}")
print(f"Accuracy Score para kernel 'sig' es: {accuracy_score(y_r_test['quality_good'], y_r_pred_sig)}")

Accuracy Score para kernel 'poly' es: 0.76
Accuracy Score para kernel 'rbf' es: 0.76
Accuracy Score para kernel 'sig' es: 0.59


In [59]:
#Accuracy Score para kernel 'rbf' es: 0.8925

In [61]:
#White
clf_poly.fit(X_w_train_scaled, y_w_train['quality_good'])
clf_rbf.fit(X_w_train_scaled, y_w_train['quality_good'])
clf_sig.fit(X_w_train_scaled, y_w_train['quality_good'])

y_w_pred_poly = clf_poly.predict(X_w_test_scaled)
y_w_pred_rbf = clf_rbf.predict(X_w_test_scaled)
y_w_pred_sig = clf_sig.predict(X_w_test_scaled)

print(f"Accuracy Score para kernel 'poly' es: {accuracy_score(y_w_test['quality_good'], y_w_pred_poly)}")
print(f"Accuracy Score para kernel 'rbf' es: {accuracy_score(y_w_test['quality_good'], y_w_pred_rbf)}")
print(f"Accuracy Score para kernel 'sig' es: {accuracy_score(y_w_test['quality_good'], y_w_pred_sig)}")

Accuracy Score para kernel 'poly' es: 0.7273469387755102
Accuracy Score para kernel 'rbf' es: 0.7877551020408163
Accuracy Score para kernel 'sig' es: 0.6938775510204082


In [62]:
#Accuracy Score para kernel 'rbf' es: 0.7877551020408163

# Exercise 6.4
Using the best SVM find the parameters that gives the best performance

'C': [0.1, 1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001]

In [65]:
# Red
accuracy = []
for c in [0.1, 1, 10, 100, 1000]:
    for g in [0.01, 0.001, 0.0001]:
        clf_rbf_cg = SVC(kernel='rbf', C=c, gamma=g)
        clf_rbf_cg.fit(X_r_train_scaled, y_r_train['quality_good'])
        y_r_pred_rbf_cg = clf_rbf_cg.predict(X_r_test_scaled)
        accuracy.append([c, g, accuracy_score(y_r_test['quality_good'], y_r_pred_rbf_cg)])
accuracy = pd.DataFrame(accuracy, columns = ['C', 'Gamma', 'Accuracy'])
print(accuracy.loc[accuracy['Accuracy'].idxmax()])

C           1000.00
Gamma          0.01
Accuracy       0.78
Name: 12, dtype: float64


In [66]:
#White
accuracy = []
for c in [0.1, 1, 10, 100, 1000]:
    for g in [0.01, 0.001, 0.0001]:
        clf_rbf_cg = SVC(kernel='rbf', C=c, gamma=g)
        clf_rbf_cg.fit(X_w_train_scaled, y_w_train['quality_good'])
        y_w_pred_rbf_cg = clf_rbf_cg.predict(X_w_test_scaled)
        accuracy.append([c, g, accuracy_score(y_w_test['quality_good'], y_w_pred_rbf_cg)])
accuracy = pd.DataFrame(accuracy, columns = ['C', 'Gamma', 'Accuracy'])
print(accuracy.loc[accuracy['Accuracy'].idxmax()])

C           1000.000000
Gamma          0.010000
Accuracy       0.787755
Name: 12, dtype: float64


# Exercise 6.5

Compare the results with other methods

In [68]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e9, solver='liblinear', multi_class='ovr')
logreg_l1 = LogisticRegression(C=0.1, penalty='l1', solver='liblinear', multi_class='ovr')
logreg_l2 = LogisticRegression(C=0.1, penalty='l2', solver='liblinear', multi_class='ovr')

In [71]:
logreg.fit(X_r_train_scaled, y_r_train['quality_good'])
logreg_l1.fit(X_r_train_scaled, y_r_train['quality_good'])
logreg_l2.fit(X_r_train_scaled, y_r_train['quality_good'])

y_r_pred_logreg = logreg.predict(X_r_test_scaled)
y_r_pred_logreg_l1 = logreg_l1.predict(X_r_test_scaled)
y_r_pred_logreg_l2 = logreg_l2.predict(X_r_test_scaled)

print(f"Accuracy Score para regresión logística es: {accuracy_score(y_r_test['quality_good'], y_r_pred_logreg)}")
print(f"Accuracy Score para regresión logística tipo Lasso es: {accuracy_score(y_r_test['quality_good'], y_r_pred_logreg_l1)}")
print(f"Accuracy Score para regresión logística truncada es: {accuracy_score(y_r_test['quality_good'], y_r_pred_logreg_l2)}")

Accuracy Score para regresión logística es: 0.745
Accuracy Score para regresión logística tipo Lasso es: 0.7425
Accuracy Score para regresión logística truncada es: 0.7325


# Regularization

# Exercise 6.6


* Train a linear regression to predict wine quality (Continous)

* Analyze the coefficients

* Evaluate the RMSE

In [74]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression(normalize=False)

In [76]:
linreg.fit(X_r_train_scaled, y_r_train['quality'])

coef_linreg_r = pd.DataFrame(linreg.coef_, columns=['Beta'])
coef_linreg_r.T

,0,1,2,3,4,5,6,7,8,9,10
Beta,0.008266,-0.102118,-0.018167,0.05811,-0.04554,0.055285,-0.1159,-0.03707,-0.068071,0.116092,0.333367


In [77]:
linreg.intercept_

5.74637907042434

In [78]:
y_r_pred_linreg = linreg.predict(X_r_test_scaled)

from sklearn import metrics
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_r_test['quality'], y_r_pred_linreg)))

RMSE: 0.6749079572603044


In [79]:
linreg.intercept_

5.74637907042434

# Exercise 6.7

* Estimate a ridge regression with alpha equals 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

In [81]:
from sklearn.linear_model import Ridge
linreg_l2_01 = Ridge(alpha=0.1)
linreg_l2_1 = Ridge(alpha=1)

In [83]:
linreg_l2_01.fit(X_r_train_scaled, y_r_train['quality'])
linreg_l2_1.fit(X_r_train_scaled, y_r_train['quality'])

coef_ridgereg_01_r = pd.DataFrame(linreg_l2_01.coef_, columns = ['Beta'])
coef_ridgereg_01_r.T

,0,1,2,3,4,5,6,7,8,9,10
Beta,0.008295,-0.102116,-0.018159,0.058112,-0.04554,0.055241,-0.11586,-0.037166,-0.06802,0.116094,0.333293


In [84]:
coef_ridgereg_1_r = pd.DataFrame(linreg_l2_1.coef_, columns = ['Beta'])
coef_ridgereg_1_r.T

,0,1,2,3,4,5,6,7,8,9,10
Beta,0.008559,-0.102105,-0.018079,0.058116,-0.045539,0.054853,-0.11551,-0.038007,-0.067563,0.116111,0.332627


# Exercise 6.8

* Estimate a lasso regression with alpha equals 0.01, 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

In [86]:
from sklearn.linear_model import Lasso
linreg_l1_001 = Lasso(alpha=0.01)
linreg_l1_01 = Lasso(alpha=0.1)
linreg_l1_1 = Lasso(alpha=1)

In [87]:
linreg_l1_001.fit(X_r_train_scaled, y_r_train['quality'])
linreg_l1_01.fit(X_r_train_scaled, y_r_train['quality'])
linreg_l1_1.fit(X_r_train_scaled, y_r_train['quality'])

coef_lasso_001_r = pd.DataFrame(linreg_l1_001.coef_, columns=['Beta'])
coef_lasso_001_r.T

,0,1,2,3,4,5,6,7,8,9,10
Beta,0.0,-0.100482,-0.003805,0.0,-0.042861,0.0,-0.072364,-0.0,-0.049017,0.10834,0.334407


In [88]:
comp_lasso_001_linreg_r = pd.DataFrame(coef_linreg_r >= coef_lasso_001_r)
comp_lasso_001_linreg_r.Beta.value_counts()

False    7
True     4
Name: Beta, dtype: int64

In [89]:
coef_lasso_01_r = pd.DataFrame(linreg_l1_01.coef_, columns=['Beta'])
coef_lasso_01_r.T

,0,1,2,3,4,5,6,7,8,9,10
Beta,0.0,-0.101703,0.0,-0.0,-0.015179,-0.0,-0.0,-0.0,-0.0,0.061603,0.241032


In [91]:
comp_lasso_01_linreg_r = pd.DataFrame(coef_linreg_r >= coef_lasso_01_r)
comp_lasso_01_linreg_r.Beta.value_counts()

False    6
True     5
Name: Beta, dtype: int64

In [92]:
coef_lasso_1_r = pd.DataFrame(linreg_l1_1.coef_, columns=['Beta'])
coef_lasso_1_r.T

,0,1,2,3,4,5,6,7,8,9,10
Beta,0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0


In [93]:
comp_lasso_1_linreg_r = pd.DataFrame(coef_linreg_r >= coef_lasso_1_r)
comp_lasso_1_linreg_r.Beta.value_counts()

False    6
True     5
Name: Beta, dtype: int64

In [94]:
y_r_pred_linreg_l1_001 = linreg_l1_001.predict(X_r_test_scaled)
y_r_pred_linreg_l1_01 = linreg_l1_01.predict(X_r_test_scaled)
y_r_pred_linreg_l1_1 = linreg_l1_1.predict(X_r_test_scaled)

print('RMSE con alpha(0.01) =', np.sqrt(metrics.mean_squared_error(y_r_test['quality'], y_r_pred_linreg_l1_001)))
print('RMSE con alpha(0.1) =', np.sqrt(metrics.mean_squared_error(y_r_test['quality'], y_r_pred_linreg_l1_01)))
print('RMSE con alpha(1) =', np.sqrt(metrics.mean_squared_error(y_r_test['quality'], y_r_pred_linreg_l1_1)))

RMSE con alpha(0.01) = 0.6789798589685276
RMSE con alpha(0.1) = 0.7019375113911199
RMSE con alpha(1) = 0.8372811298654157


# Exercise 6.9

* Create a binary target

* Train a logistic regression to predict wine quality (binary)

* Analyze the coefficients

* Evaluate the f1score

In [96]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e9, solver='liblinear', multi_class='ovr')

In [98]:
logreg.fit(X_r_train_scaled, y_r_train['quality_good'])

coef_logreg_r = pd.DataFrame(logreg.coef_, index=['Beta'])
coef_logreg_r

,0,1,2,3,4,5,6,7,8,9,10
Beta,0.057478,-0.300557,-0.141402,0.192292,-0.122578,0.336114,-0.688245,-0.071444,-0.108711,0.360799,1.025323


In [99]:
y_r_pred_logreg = logreg.predict(X_r_test_scaled)

from sklearn.metrics import f1_score

f1_score(y_r_test['quality_good'], y_r_pred_logreg)

0.7723214285714286

# Exercise 6.10

* Estimate a regularized logistic regression using:
* C = 0.01, 0.1 & 1.0
* penalty = ['l1, 'l2']
* Compare the coefficients and the f1score

In [100]:
score = []
def logreg_fun(X_train, y_train, X_test, y_test):
    for p in ['l1', 'l2']:
        for c in [0.01, 0.1, 1.0]:
            logreg = LogisticRegression(C=c, penalty=p, solver='liblinear', multi_class='ovr')
            logreg.fit(X_train, y_train)
            y_pred_logreg = logreg.predict(X_test)
            score.append([p, c, f1_score(y_test, y_pred_logreg)])

In [103]:
score = []
logreg_fun(X_r_train_scaled, y_r_train['quality_good'], X_r_test_scaled, y_r_test['quality_good'])
score = pd.DataFrame(score, columns=['Penalty', 'C', 'F1_Score'])
print(score.loc[score['F1_Score'].idxmax()])

Penalty           l1
C                  1
F1_Score    0.773333
Name: 2, dtype: object


In [104]:
score = []
logreg_fun(X_w_train_scaled, y_w_train['quality_good'], X_w_test_scaled, y_w_test['quality_good'])
score = pd.DataFrame(score, columns=['Penalty', 'C', 'F1_Score'])
print(score.loc[score['F1_Score'].idxmax()])

Penalty           l2
C                0.1
F1_Score    0.822978
Name: 4, dtype: object
